In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/THAThack_356', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

           id                date  latitude  longitude  week
0         222 2015-04-26 05:40:00  4.865703  45.756363    17
2         697 2015-03-29 01:33:00  4.868730  45.764233    13
3         653 2015-04-08 05:40:00  4.866808  45.756482    15
5         972 2015-04-01 23:51:00  4.986643  45.699552    14
8         192 2015-03-18 16:44:00  4.884393  45.768457    12
...       ...                 ...       ...        ...   ...
34551843  329 2015-03-11 13:16:00  4.872078  45.782602    11
34551844  841 2015-03-20 15:21:00  4.282412  45.565612    12
34551846  186 2015-03-15 08:59:00  6.396280  46.237458    11
34551847  141 2015-03-11 04:11:00  4.872033  45.784828    11
34551848  855 2015-05-06 13:44:00  4.871782  45.784938    19

[20159188 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['10' '1000' '1001' '1002' '1005' '1010' '1014' '1015' '1017' '1018'
 '1019' '1020' '1024' '1028' '103' '1032' '1033' '1035' '1042' '1051'
 '1054' '106' '1060' '109' '113' '114' '115' '120' '121' '13' '130' '134'
 '136' '137' '140' '141' '143' '144' '146' '15' '150' '152' '160' '164'
 '165' '168' '169' '176' '18' '180' '181' '182' '184' '185' '186' '189'
 '191' '192' '193' '196' '197' '200' '201' '205' '207' '211' '213' '214'
 '215' '222' '225' '228' '23' '232' '234' '24' '240' '245' '246' '249'
 '251' '254' '255' '257' '258' '259' '260' '262' '265' '269' '271' '272'
 '278' '279' '28' '281' '285' '289' '29' '290' '292' '296' '303' '304'
 '307' '31' '314' '318' '319' '32' '320' '324' '329' '33' '330' '333'
 

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [12]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])), 4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600), 1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)), 1/2)
            
            list_distance.append(distance*delta_ratio*delta_hour*delta_time) #*delta_count*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

306
{'1': {'2015-10': ['168'], '2015-11': ['978'], '2015-12': ['184'], '2015-13': ['810'], '2015-14': ['676'], '2015-15': ['70'], '2015-16': ['425'], '2015-17': ['265'], '2015-18': ['485'], '2015-19': ['755'], '2015-20': ['258']}, '2': {'2015-10': ['34'], '2015-11': ['694'], '2015-12': ['342'], '2015-13': ['904'], '2015-14': ['358'], '2015-15': ['368'], '2015-16': ['289'], '2015-17': ['527'], '2015-18': ['952'], '2015-19': ['468'], '2015-20': ['152']}, '107': {'2015-10': ['524'], '2015-11': ['1010'], '2015-12': ['681']}, '98': {'2015-10': ['607'], '2015-11': ['76']}, '21': {'2015-10': ['292'], '2015-11': ['28']}, '5': {'2015-10': ['879'], '2015-11': ['463']}, '9': {'2015-10': ['980'], '2015-11': ['193'], '2015-12': ['285'], '2015-13': ['1033'], '2015-14': ['103'], '2015-15': ['793'], '2015-16': ['87'], '2015-19': ['707']}, '6': {'2015-10': ['579'], '2015-11': ['476'], '2015-12': ['1018'], '2015-13': ['867'], '2015-14': ['510'], '2015-15': ['653'], '2015-16': ['984']}, '7': {'2015-10': 

In [8]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "168"
        ],
        "2015-11": [
            "978"
        ],
        "2015-12": [
            "184"
        ],
        "2015-13": [
            "810"
        ],
        "2015-14": [
            "676"
        ],
        "2015-15": [
            "70"
        ],
        "2015-16": [
            "425"
        ],
        "2015-17": [
            "265"
        ],
        "2015-18": [
            "485"
        ],
        "2015-19": [
            "755"
        ],
        "2015-20": [
            "258"
        ]
    },
    "2": {
        "2015-10": [
            "34"
        ],
        "2015-11": [
            "694"
        ],
        "2015-12": [
            "342"
        ],
        "2015-13": [
            "904"
        ],
        "2015-14": [
            "358"
        ],
        "2015-15": [
            "368"
        ],
        "2015-16": [
            "289"
        ],
        "2015-17": [
            "527"
        ],
        "2015-18": [
 